In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
import cv2

# Ścieżki do folderów z danymi
train_path = "./images/train-kopia/"
val_path = "./images/validation/"

# Rozmiar obrazów
pic_size = 48

# Lista emocji (kategorii)
emotions = os.listdir(train_path)

# Przygotowanie danych treningowych
train_data = []
train_labels = []

for emotion in emotions:
    emotion_path = os.path.join(train_path, emotion)
    if not os.path.isdir(emotion_path):
        continue

    for img_name in os.listdir(emotion_path):
        if img_name == '.DS_Store':
            continue

        img_path = os.path.join(emotion_path, img_name)
        img = load_img(img_path, target_size=(pic_size, pic_size), color_mode='grayscale')
        img_array = img_to_array(img)
        train_data.append(img_array)
        train_labels.append(emotions.index(emotion))  # Przypisanie etykiety numerycznej

# Przygotowanie danych walidacyjnych
val_data = []
val_labels = []

for emotion in emotions:
    emotion_path = os.path.join(val_path, emotion)
    if not os.path.isdir(emotion_path):
        continue

    for img_name in os.listdir(emotion_path):
        if img_name == '.DS_Store':
            continue

        img_path = os.path.join(emotion_path, img_name)
        img = load_img(img_path, target_size=(pic_size, pic_size), color_mode='grayscale')
        img_array = img_to_array(img)
        val_data.append(img_array)
        val_labels.append(emotions.index(emotion))  # Przypisanie etykiety numerycznej

# Normalizacja danych
train_data = np.array(train_data, dtype="float32") / 255.0
train_labels = np.array(train_labels)
val_data = np.array(val_data, dtype="float32") / 255.0
val_labels = np.array(val_labels)

# One-hot encoding etykiet
train_labels = to_categorical(train_labels, len(emotions))
val_labels = to_categorical(val_labels, len(emotions))

# Tworzenie modelu CNN
model = Sequential()

# Warstwa konwolucyjna 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(pic_size, pic_size, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Warstwa konwolucyjna 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Warstwa konwolucyjna 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Spłaszczenie
model.add(Flatten())

# Warstwa gęsta (fully connected)
model.add(Dense(128, activation='relu'))
model.add(Dense(len(emotions), activation='softmax'))  # Wyjście: liczba emocji

# Kompilowanie modelu
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(train_data, train_labels, epochs=20, batch_size=32, validation_data=(val_data, val_labels))

# Ocena modelu na danych walidacyjnych
val_loss, val_accuracy = model.evaluate(val_data, val_labels)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


659/659 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.3816 - loss: 1.3200 - val_accuracy: 0.5329 - val_loss: 1.0617
Epoch 2/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.5588 - loss: 1.0363 - val_accuracy: 0.5907 - val_loss: 0.9700
Epoch 3/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6180 - loss: 0.9160 - val_accuracy: 0.6257 - val_loss: 0.8978
Epoch 4/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6479 - loss: 0.8537 - val_accuracy: 0.6348 - val_loss: 0.8824
Epoch 5/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6662 - loss: 0.8040 - val_accuracy: 0.6416 - val_loss: 0.8644
Epoch 6/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7002 - loss: 0.7412 - val_accuracy: 0.6506 - val_loss: 0.8614
Epoch 7/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7289 - loss: 0.6858 - val_accuracy: 0.6551 - val_loss: 0.8766
Epoch 8/20
659/659 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7500 - loss: 0.6278 - val_accuracy:

In [ ]:
import time
import cv2

pic_size = 48


def predict_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Przekształcanie obrazu na skalę szarości
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # Ładowanie klasyfikatora twarzy
    
    # Zwiększenie wartości scaleFactor i minNeighbors, aby zmniejszyć fałszywe wykrycia
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=4, minSize=(30, 30))  # Wykrywanie twarzy
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]  # Wycinanie twarzy
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Konwertowanie twarzy na skalę szarości
        face = cv2.resize(face, (pic_size, pic_size))  # Zmiana rozmiaru twarzy
        face = face.reshape(1, pic_size, pic_size, 1)  # Reshape twarzy do formatu (1, 48, 48, 1)
        face = face / 255.0  # Normalizacja obrazu
        
        # Przewidywanie emocji
        prediction = model.predict(face)
        max_index = np.argmax(prediction[0])
        emotion_predicted = emotions[max_index]
        
        # Wyświetlanie wyniku
        cv2.putText(frame, emotion_predicted, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return frame

# Uruchomienie kamery
cap = cv2.VideoCapture(0)

# Rozpoczęcie liczenia czasu
start_time = time.time()

while True:
    ret, frame = cap.read()  # Odczytanie obrazu z kamery
    if not ret:
        break
    
    # Przewidywanie emocji na obrazie
    frame = predict_emotion(frame)
    
    # Wyświetlanie obrazu z wykrytymi emocjami
    cv2.imshow('Emotion Recognition', frame)
    
    # Sprawdzanie, czy upłynęło 30 sekund
    elapsed_time = time.time() - start_time
    if elapsed_time > 30:
        break  # Przerwij po 30 sekundach
    
    # Zatrzymanie kamery po naciśnięciu 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  # Zwolnienie kamery
cv2.destroyAllWindows()  # Zamykanie okien

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


2024-12-16 17:52:59.829 python[14799:1489217] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-16 17:52:59.829 python[14799:1489217] +[IMKInputSession subclass]: chose IMKInputSession_Modern


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

: 